# Requirements

In [1]:
import pandas as pd
import numpy as np

import datetime
import pytz
import requests
from pandas.io.json import json_normalize

import pickle

import matplotlib.pyplot as plt
%matplotlib inline

# Dataset

In [2]:
IEX_API_URL_TEMPLATE = 'https://api.iextrading.com/1.0/stock/{}/chart/{}'

HIST_5Y = '5y'
HIST_1Y = '1y'
HIST_1M = '1m'

djia_tickers = [
    'BA',   'PFE', 'MCD', 'WMT', 'KO',   'MRK',  'HD',   'V',   'JNJ',  'VZ',
    'CSCO', 'AXP', 'TRV', 'DIS', 'MSFT', 'UNH',  'DWDP', 'CAT', 'AAPL', 'UTX',
    'MMM',  'JPM', 'IBM', 'GS',  'XOM',  'INTC', 'NKE',  'CVX', 'PG',   'WBA' ]

In [6]:
def format_dataset(df):
    df['year'] = df.date.dt.year
    df['month'] = df.date.dt.month
    df['day'] = df.date.dt.day
    df['week'] = df.date.dt.week
    df['dayofweek'] = df.date.dt.dayofweek
    df['dayofyear'] = df.date.dt.dayofyear
    df['timestamp'] = df.date.values.astype(np.int64)

    return df.drop(['label',
                    'change', 'changeOverTime', 'changePercent',
                    'high', 'low', 'open',
                    'unadjustedVolume', 'volume', 'vwap'],
                   axis=1)


def get_dataset_from_ticker(ticker_symbol, hist_period=HIST_5Y):
    r = requests.get(url = IEX_API_URL_TEMPLATE.format(ticker_symbol.lower(), hist_period))
    df = json_normalize(r.json())

    df.date = pd.to_datetime(df.date, format='%Y-%m-%d')
    df.insert(loc=0, column='symbol', value=ticker_symbol)

    return format_dataset(df)


def get_djia_dataset(hist_period=HIST_5Y):
    df = None

    for ticker_symbol in djia_tickers:
        if df is None:
            df = get_dataset_from_ticker(ticker_symbol, hist_period=hist_period)
        else:
            df = df.append(get_dataset_from_ticker(ticker_symbol, hist_period))

    return df


def update_djia_dataset(djia_ds):
    last_recorded_day = max(djia_ds.date)
    today = datetime.datetime.now()
    days_to_update = (today - last_recorded_day).days

    hist_period = HIST_5Y

    if days_to_update < 1:
        return djia_ds
    elif days_to_update < 28:
        hist_period = HIST_1M
    elif days_to_update < 365:
        hist_period = HIST_1Y

    last_djia_ds = get_djia_dataset(hist_period)

    return djia_ds.append(last_djia_ds).drop_duplicates(['symbol', 'date'], keep='last')


def save_dataset_to_file(ds, file_name):
    with open(file_name, 'wb') as fp:
        pickle.dump(ds, fp)


def load_dataset_from_file(file_name):
    with open(file_name, 'rb') as fp:
        return pickle.load(fp)

# Trading days

In [7]:
market_holidays = []

def load_market_holidays(market_holidays_file='market_holidays.txt'):
    market_holidays.clear()

    with open(market_holidays_file) as f:
        lines = f.readlines()

    for line in lines:
        market_holidays.append(datetime.datetime.strptime(line.strip(), '%Y-%m-%d'))

load_market_holidays()

In [9]:
def is_trading_day(day):
    day_of_week = day.weekday()
    if day_of_week == 5 or day_of_week == 6 or day in market_holidays:
        return False
    else:
        return True


def get_trading_days_in_range(start_date, end_date):
    trading_days = []

    current_day = start_date
    while (current_day <= end_date):
        if is_trading_day(current_day):
            trading_days.append(current_day)
        current_day += datetime.timedelta(days=1)

    return trading_days